In [8]:
import os
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field
from openai import OpenAI
import requests
import logging
import json
from collections import defaultdict
from datetime import datetime

In [3]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [4]:
model = "llama-3.3-70b-versatile"

In [5]:

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "Write a limerick about the Python programming language.",
        },
    ],
)

In [6]:
completion.model_dump()

{'id': 'chatcmpl-8cff838a-c9b7-4701-a859-bf3795022f47',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'There once was a Python so fine,\nWhose code was easy to design.\nIt ran with great pace,\nAnd a gentle face,\nAnd its libraries were truly divine.',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': None}}],
 'created': 1743039233,
 'model': 'llama-3.3-70b-versatile',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_8dd9fca28c',
 'usage': {'completion_tokens': 35,
  'prompt_tokens': 52,
  'total_tokens': 87,
  'completion_tokens_details': None,
  'prompt_tokens_details': None,
  'queue_time': 0.10126533,
  'prompt_time': 0.002585022,
  'completion_time': 0.127272727,
  'total_time': 0.129857749},
 'x_groq': {'id': 'req_01jqakqz9qe3kvbs4j553vd5c9'}}

In [9]:
class check_prompt(BaseModel):
    """First LLM call: Extract basic accomodation information"""

    # description: str = Field(description="Raw description of the event")
    is_accomodation_search: bool = Field(
        description="Is the text a query related to finding an accomodation?"
    )

In [36]:
def check_accomodation_request(input_prompt: str):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"""
                    You are an assistant for finding accomodation. 
                    You are allowed to answer only to queries related to accomodation search.
                    In case of other requests, you must decline.
                    In this context, Analyze if the text describes a query for finding accomodation.
                    Say yes if the text describes a query for finding accomodation, else say no.
                    Output should be in json format.
                    {json.dumps(check_prompt.model_json_schema(), indent=2)}
                    """ 
            },
            {"role": "user", "content": input_prompt},
        ],
        response_format={"type": "json_object"}
    )

    return check_prompt.model_validate_json(completion.choices[0].message.content)

In [37]:
user_input = "Suggest me some hotels in Bengaluru for staying from 27th March 2025 and 31st March 2025"

In [38]:
check_accomodation_request(user_input)

check_prompt(is_accomodation_search=True)